In [1]:
import numpy as np
import torch

#check GPU
print('CUDA available: {}'.format(torch.cuda.is_available()))
print('Current GPU: {}'.format(torch.cuda.get_device_name(torch.cuda.current_device())))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

CUDA available: True
Current GPU: GeForce RTX 2080 SUPER


In [2]:
import deepchem as dc

loaded, datasets, transformers = dc.utils.load_dataset_from_disk('data/combined/amg')
train_dataset, valid_dataset, test_dataset = datasets
print('train/val/test split: {}/{}/{}'.format(
    len(train_dataset), len(valid_dataset), len(test_dataset)))

num_node_features = train_dataset.X[0].num_node_features
num_edge_features = train_dataset.X[0].num_edge_features
num_classes = train_dataset.y[0].shape[-1]
print(num_node_features, num_classes)

train/val/test split: 483/60/61
79 1


In [3]:
from torch_geometric.data import Data, DataLoader

def get_data_loader(dc_dataset, batch_size=64, shuffle=True):
    ds = [x.to_pyg_graph() for x in dc_dataset.X]
    for i in range(len(ds)):
        ds[i].y = torch.from_numpy(dc_dataset.y[i].reshape(1, -1))
        ds[i].w = torch.from_numpy(dc_dataset.w[i].reshape(1, -1))
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)

test_loader = get_data_loader(test_dataset, batch_size=64)

RDKit WARNING: [02:32:56] Enabling RDKit 2019.09.3 jupyter extensions


In [4]:
from sklearn.metrics import roc_auc_score

def test(model, loader):
    model.eval()

    outs = []
    ys = []
    for data in loader:  # Iterate in batches over the training/test dataset.
        data = data.to(device)
        
        #out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        out = model(data.x, data.edge_index, data.edge_attr, data.batch)
        
        outs.append(out.detach().cpu().numpy())
        ys.append(data.y.detach().cpu().numpy())
    
    pred = np.concatenate(outs, axis=0)
    y = np.concatenate(ys, axis=0)
    
    score = roc_auc_score(y, pred)
    return score

#model = torch.load('trained_models/AFP_tl_amg_0.8561')
model = torch.load('trained_models/AFP_tl_amg_test')
te_score = test(model, test_loader)
print('Test: {:.5f}'.format(te_score))

Test: 0.88095


In [5]:
from featurizer import MolGraphConvFeaturizer

featurizer = MolGraphConvFeaturizer(
    use_edges=True, use_chirality=True, use_partial_charge=True
)

In [6]:
from rdkit import Chem
from utils import validate_mols

def get_valid_smiles(data):
    smiles = list(data['smiles'])
    mols = [Chem.MolFromSmiles(s) for s in smiles]
    valid_mols, valid_index = validate_mols(mols)
    valid_smiles = [Chem.MolToSmiles(mol) for mol in valid_mols]
    return valid_smiles, valid_index

def get_valid_mols(data):
    smiles = list(data['smiles'])
    mols = [Chem.MolFromSmiles(s) for s in smiles]
    valid_mols, valid_index = validate_mols(mols)
    return valid_mols, valid_index

def get_valid_features(molecules, featurizer):
    valid_features = []
    valid_index = []
    for i, mol in enumerate(molecules):
        try:
            valid_features.append(featurizer._featurize(mol))
            valid_index.append(i)
        except Exception as e:
            print("Failed to featurize datapoint %d, %s.", i, Chem.MolToSmiles(mol))
            print("Exception message: {}".format(e))

    return np.asarray(valid_features), np.array(valid_index)

def featurize_data(data, featurizer):
    valid_smiles, valid_index = get_valid_smiles(data)
    valid_mols, valid_index = get_valid_mols(data)
    valid_data = data.iloc[valid_index]
    valid_features, valid_index = get_valid_features(valid_mols, featurizer)
    valid_data = valid_data.iloc[valid_index]
    return valid_features, valid_data

In [7]:
import pandas as pd

fda_approved_data = pd.read_csv('data/tl/approved_drugs_valid_parent.csv')
fda_approved_data

,chembl_id,pref_name,smiles
0,CHEMBL2,PRAZOSIN,COc1cc2nc(nc(N)c2cc1OC)N3CCN(CC3)C(=O)c4occc4
1,CHEMBL3,NICOTINE,CN1CCC[C@H]1c2cccnc2
2,CHEMBL4,OFLOXACIN,CC1COc2c(N3CCN(C)CC3)c(F)cc4C(=O)C(=CN1c24)C(=O)O
3,CHEMBL5,NALIDIXIC ACID,CCN1C=C(C(=O)O)C(=O)c2ccc(C)nc12
4,CHEMBL6,INDOMETHACIN,COc1ccc2c(c1)c(CC(=O)O)c(C)n2C(=O)c3ccc(Cl)cc3
...,...,...,...
2294,CHEMBL3833405,INOSINE PRANOBEX,CC(O)CN(C)C.CC(O)CN(C)C.CC(O)CN(C)C.OC[C@H]1O[...
2295,CHEMBL3833406,PAPAVERETUM,COc1ccc(Cc2nccc3cc(OC)c(OC)cc23)cc1OC.COc4ccc5...
2296,CHEMBL3833408,MAGALDRATE,[OH-].[OH-].[OH-].[OH-].[OH-].[OH-].[OH-].[OH-...
2297,CHEMBL3833409,HYDROTALCITE,[OH-].[OH-].[OH-].[OH-].[OH-].[OH-].[OH-].[OH-...


In [8]:
valid_fda_approved_features, valid_fda_approved_data = featurize_data(
    fda_approved_data, featurizer)

Failed to featurize datapoint %d, %s. 607 [I-].[K+]
Exception message: zero-size array to reduction operation maximum which has no identity
Failed to featurize datapoint %d, %s. 815 [F-].[Na+]
Exception message: zero-size array to reduction operation maximum which has no identity
Failed to featurize datapoint %d, %s. 1458 O
Exception message: zero-size array to reduction operation maximum which has no identity
Failed to featurize datapoint %d, %s. 1462 N
Exception message: zero-size array to reduction operation maximum which has no identity
Failed to featurize datapoint %d, %s. 1541 [Cl-].[Na+]
Exception message: zero-size array to reduction operation maximum which has no identity
Failed to featurize datapoint %d, %s. 1565 [Ca+2].[Cl-].[Cl-]
Exception message: zero-size array to reduction operation maximum which has no identity
Failed to featurize datapoint %d, %s. 1566 [Cl-].[Cl-].[Zn+2]
Exception message: zero-size array to reduction operation maximum which has no identity
Failed to 

In [9]:
def predict(model, loader):
    model.eval()

    outs = []
    for data in loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.edge_attr, data.batch)
        outs.append(torch.sigmoid(out).detach().cpu().numpy())
    
    pred = np.concatenate(outs, axis=0)
    return pred

loader = DataLoader([x.to_pyg_graph() for x in valid_fda_approved_features], 
           batch_size=16, shuffle=False)
pred = predict(model, loader)

In [10]:
res_fda_approved_data = valid_fda_approved_data.copy()
res_fda_approved_data.insert(3, 'p', pred)
res_fda_approved_data_sorted = res_fda_approved_data.sort_values(by=['p'], ascending=False)
res_fda_approved_data_sorted

,chembl_id,pref_name,smiles,p
236,CHEMBL558,MEXILETINE,CC(N)COc1c(C)cccc1C,0.906852
945,CHEMBL14687,PROPANOL,CCCO,0.904993
1244,CHEMBL312448,XYLOMETAZOLINE,Cc1cc(cc(C)c1CC2=NCCN2)C(C)(C)C,0.904881
1175,CHEMBL256087,MENTHOL,CC(C)C1CCC(C)CC1O,0.900934
1765,CHEMBL1201338,CYCLOPENTOLATE,CN(C)CCOC(=O)C(c1ccccc1)C2(O)CCCC2,0.900472
...,...,...,...,...
1024,CHEMBL53463,DOXORUBICIN,COc1cccc2C(=O)c3c(O)c4C[C@](O)(C[C@H](O[C@H]5C...,0.044964
110,CHEMBL178,DAUNORUBICIN,COc1cccc2C(=O)c3c(O)c4C[C@](O)(C[C@H](O[C@H]5C...,0.044499
1466,CHEMBL1165342,VINCAMINE,CC[C@@]12CCCN3CCc4c([C@H]13)n(c5ccccc45)[C@](O...,0.044104
597,CHEMBL1117,IDARUBICIN,C[C@@H]1O[C@H](C[C@H](N)[C@@H]1O)O[C@H]2C[C@@]...,0.043911


In [19]:
def check_data(train_dataset, valid_dataset, test_dataset, res_data):
    data = res_data.copy()
    data.insert(len(data.columns), 'dataset', np.full_like(data['p'], np.nan))
    data.insert(len(data.columns), 'label', np.full_like(data['p'], np.nan))
    
    dataset_col_loc = len(data.columns) - 2
    labelcol_loc = len(data.columns) - 1
    
    for i in range(len(data)):
        curr_smiles = data.iloc[i]['smiles']
        
        for j, s in enumerate(train_dataset.ids):
            if s == curr_smiles:
                data.iloc[i, dataset_col_loc] = 'train'
                if train_dataset.y[j][0] == 1.:
                    data.iloc[i, labelcol_loc] = 1
                else:
                    data.iloc[i, labelcol_loc] = 0
                    
        for j, s in enumerate(valid_dataset.ids):
            if s == curr_smiles:
                data.iloc[i, dataset_col_loc] = 'validation'
                if valid_dataset.y[j][0] == 1.:
                    data.iloc[i, labelcol_loc] = 1
                else:
                    data.iloc[i, labelcol_loc] = 0
                    
        for j, s in enumerate(test_dataset.ids):
            if s == curr_smiles:
                data.iloc[i, dataset_col_loc] = 'test'
                if test_dataset.y[j][0] == 1.:
                    data.iloc[i, labelcol_loc] = 1
                else:
                    data.iloc[i, labelcol_loc] = 0
                    
    return data

res_fda_data = check_data(
    train_dataset, valid_dataset, test_dataset, res_fda_approved_data_sorted)

In [21]:
res_fda_data_sorted = res_fda_data.sort_values(by=['dataset', 'p'], ascending=False)
res_fda_data_sorted

,chembl_id,pref_name,smiles,p,dataset,label
1741,CHEMBL1201293,ACAMPROSATE,CC(=O)NCCCS(=O)(=O)O,0.765112,validation,0.0
2141,CHEMBL2141296,IXAZOMIB,CC(C)C[C@H](NC(=O)CNC(=O)c1cc(Cl)ccc1Cl)B(O)O,0.302787,validation,0.0
194,CHEMBL497,CLIOQUINOL,Oc1c(I)cc(Cl)c2cccnc12,0.108132,validation,0.0
408,CHEMBL807,MEMANTINE,CC12CC3CC(C)(C1)CC(N)(C3)C2,0.884105,train,1.0
21,CHEMBL27,PROPRANOLOL,CC(C)NCC(O)COc1cccc2ccccc12,0.880238,train,0.0
...,...,...,...,...,...,...
1024,CHEMBL53463,DOXORUBICIN,COc1cccc2C(=O)c3c(O)c4C[C@](O)(C[C@H](O[C@H]5C...,0.044964,NaN,NaN
110,CHEMBL178,DAUNORUBICIN,COc1cccc2C(=O)c3c(O)c4C[C@](O)(C[C@H](O[C@H]5C...,0.044499,NaN,NaN
1466,CHEMBL1165342,VINCAMINE,CC[C@@]12CCCN3CCc4c([C@H]13)n(c5ccccc45)[C@](O...,0.044104,NaN,NaN
597,CHEMBL1117,IDARUBICIN,C[C@@H]1O[C@H](C[C@H](N)[C@@H]1O)O[C@H]2C[C@@]...,0.043911,NaN,NaN


In [22]:
res_fda_data_sorted.to_csv('results/amg_fda_pred-210725.csv', index=False)